In [1]:
import pandas as pd

# Load the pool day data
pool_day_data_df = pd.read_csv('pool_day_data.csv', low_memory=False)  # Addressing DtypeWarning

In [14]:
# Convert Unix timestamp to datetime
# Assuming 'date' column is in Unix timestamp format (seconds since epoch)
pool_day_data_df['date'] = pd.to_datetime(pool_day_data_df['date'], unit='s')

In [3]:
# Load the pool day data
pools_data_df = pd.read_csv('pools.csv', low_memory=False)  # Addressing DtypeWarning

print(len(pools_data_df))

19234


In [11]:
# Get pools that have WETH

WETH = "WETH"  # Replace with actual WETH address
filtered_pools = pools_data_df[((pools_data_df['token0.symbol'] == WETH) | (pools_data_df['token1.symbol'] == WETH))].copy()
filtered_pools.sort_values(by=['id'], inplace=True)

# Extract pool IDs
weth_pool_ids = filtered_pools['id'].unique()

print(len(weth_pool_ids))

13493


In [12]:
# Get pools that had more than 5000 TVL at one point

# Filter for days where TVL > $5000
high_tvl_days = pool_day_data_df[pool_day_data_df['totalValueLockedUSD'] > 5000]

# Get unique pool IDs from these rows
high_tvl_pool_ids = high_tvl_days['pool.id'].unique()

print(len(high_tvl_pool_ids))

7641


In [13]:
# Find intersection of pool IDs
intersection_pool_ids = set(high_tvl_pool_ids).intersection(set(weth_pool_ids))

# Print the number of pools in the intersection
print(f"Number of pools with > $5000 TVL and involving WETH: {len(intersection_pool_ids)}")

Number of pools with > $5000 TVL and involving WETH: 5272


In [16]:
# Filter pool_day_data_df to include only the pools in the intersection
filtered_pool_day_data = pool_day_data_df[pool_day_data_df['pool.id'].isin(intersection_pool_ids)]

print(filtered_pool_day_data.head())  # Print first few rows to check
print(f"Number of entries in filtered pool day data: {len(filtered_pool_day_data)}")


            close       date                  feeGrowthGlobal0X128  \
389  28792.894826 2023-03-16  154042374028593464275858183529511242   
390  26778.491102 2023-03-17  172590147036242261772466326558608228   
391  33242.284462 2023-03-18  178112612372930278976686998149752369   
392  34927.776534 2023-03-19  200383769966655764133728242873124551   
393  34031.117899 2023-03-20  209002689798712475513256353257342701   

                         feeGrowthGlobal1X128       feesUSD          high  \
389  3500190821504896744612912852039898318463  10882.471209  3.402568e+38   
390  3924761051846481640284323769347360814245   2029.219302  5.138232e-05   
391  4160622340317159998835969006041881367836    764.707837  3.909948e-05   
392  4918557819042234731605056519108076409604   1341.174786  4.044482e-05   
393  5203913553972883546527313527848417477521    480.124937  3.873842e-05   

                                                   id  \
389  0x000c0d31f6b7cecde4645eef0c4ec6a492659d62-19432   
39

In [19]:
def detect_rugpulls(dataframe):
    rugpulls = []
    for pool_id, group in dataframe.groupby('pool.id'):
        group = group.sort_values(by='date')  # Ensure data is sorted by date
        rugpull_detected = False
        for i in range(1, len(group)):
            previous_day_tvl = group.iloc[i-1]['totalValueLockedUSD']
            current_day_tvl = group.iloc[i]['totalValueLockedUSD']
            if current_day_tvl < 0.01 * previous_day_tvl and previous_day_tvl > 5000:  # Check for rugpull condition
                rugpulls.append({
                    'pool_id': pool_id,
                    'date': group.iloc[i]['date'],
                    'previous_day_tvl': previous_day_tvl,
                    'current_day_tvl': current_day_tvl,
                    'rugpull': True
                })
                rugpull_detected = True
                break  # Only consider the first such drop

        if not rugpull_detected:
            # If no rugpull detected, add a record with rugpull as False
            # You can choose the appropriate values for date, previous_day_tvl, and current_day_tvl
            rugpulls.append({
                'pool_id': pool_id,
                'date': None,  # or the last date in group, or another placeholder
                'previous_day_tvl': None,  # or another placeholder
                'current_day_tvl': None,  # or another placeholder
                'rugpull': False
            })
            
    return pd.DataFrame(rugpulls)


rugpulls_df = detect_rugpulls(filtered_pool_day_data)
rugpulls_df.to_csv('rugpulls.csv', index=False)

In [22]:
print(len(rugpulls_df))
print(rugpulls_df.head())
print(len(rugpulls_df[rugpulls_df['rugpull'] == True]))

5272
                                      pool_id       date  previous_day_tvl  \
0  0x000c0d31f6b7cecde4645eef0c4ec6a492659d62        NaT               NaN   
1  0x000ea4a83acefdd62b1b43e9ccc281f442651520        NaT               NaN   
2  0x000fedac8a4c7f2c291c5bca0fd244e17e27c763 2023-05-19      25271.091389   
3  0x0025ade782cc2b2415d1e841a8d52ff5dce33dfe        NaT               NaN   
4  0x005e3dc62b7a269bef2a7d06e06cc0c991375c6f 2022-05-12      12441.980803   

   current_day_tvl  rugpull  
0              NaN    False  
1              NaN    False  
2     7.200000e-15     True  
3              NaN    False  
4     3.073691e+01     True  
1135


In [25]:
# Select only necessary columns from pools_df
pools_subset_df = pools_data_df[['id', 'token0.id', 'token0.name', 'token0.symbol', 'token1.id', 'token1.name', 'token1.symbol']]

# Merge rugpulls_df with the subset of pools data
merged_df = rugpulls_df.merge(pools_subset_df, left_on='pool_id', right_on='id', how='left')

# Drop the extra 'id' column from the merge
merged_df.drop(columns=['id'], inplace=True)

merged_df.to_csv('rugpulls_with_token_info.csv', index=False)


In [26]:
import pandas as pd

WETH_ID = "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2"

# Load the rugpulls data with token info
rugpulls_df = pd.read_csv('rugpulls_with_token_info.csv')

# Get unique token IDs from both token0 and token1
token_ids = set(rugpulls_df['token0.id']).union(set(rugpulls_df['token1.id']))

# Remove WETH token ID from the set
token_ids.discard(WETH_ID)  # Replace "WETH" with actual WETH token ID


In [32]:
import requests

def get_token_data(token_id, api_key='EK-fewp2-ycMSQm7-3Qj3b'):
    url = f'https://api.ethplorer.io/getTokenInfo/{token_id}?apiKey={api_key}'
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.json()
        else:
            print(f"Failed to fetch data for token {token_id}, status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"Error fetching data for token {token_id}: {e}")
        return None



In [44]:
def fetch_token_data_in_batches(token_ids, batch_size=500, api_key='EK-fewp2-ycMSQm7-3Qj3b'):
    batched_token_data = []

    total_batches = len(token_ids) // batch_size + (1 if len(token_ids) % batch_size else 0)
    for i in range(total_batches):
        batch_start = i * batch_size
        batch_end = batch_start + batch_size
        current_batch = token_ids[batch_start:batch_end]
        
        for token_id in current_batch:
            data = get_token_data(token_id, api_key)
            batched_token_data.append(data)
        
        print(f"Completed batch {i+1} of {total_batches}")

    return batched_token_data

# Convert the set to a list
token_ids_list = list(token_ids)
print(len(token_ids_list))
# token_data = fetch_token_data_in_batches(token_ids_list)

4605


In [42]:
import pandas as pd

print(len(token_data))
# Check the first few elements of token_data
token_data_dicts = []
for i, item in enumerate(token_data):
    if (type(item) != dict):
        print(item, i)
    else:
        token_data_dicts.append(item)
token_df = pd.DataFrame(token_data_dicts)
token_df.to_csv('token_data.csv', index=False)

4605
None 1582
